# Día 2 Scrapy y Filtrado de datos
El día de ayer tuvimos un primer vistazo de lo complicado que puede ser realizar el scrappeo de páginas que desconocemos. La verdad es algo difícil recorrer esa estructura con facilidad y utilizando beautifulsoup tenemos que navegar muchos niveles para después aplicar métodos para organizar, filtrar y exportar esa información. A continuación vamos a presentar como Scrapy puede ser utilizado para facilitar el trabajo de obtención de información y como podemos usar funciones nativas de python para filtrar listas en caso de ser necesario.

# Scrapy
Scrapy se define como "an open source framework for web crawling". Aunque esta se utilizó principalmente para scrapping también se puede usar en conjunto con los API's para extraer información. Ahora, el concepto se le conoce como web crawling ya que a diferencia de beautifulsoup4, nostoros estamos mandando "spiders". Es decir, ya está mandando una serie de instrucciones con las cuales va a buscar información específica dentro de la página de internet lo cual nos da mucha flexibilidad al aplicar esto.

<img src="img/scrapyim.png" />

# ¿Como podemos usar Scrapy?
Hay dos formas en las cuales podemos comenzar a usar Scrapy; la primera es directamente por medio de la terminal lo cual es análogo a usar la terminal de Python. Esto sirve para proyectos temporales que no queremos implementar después o solo para realizar ciertas pruebas con las páginas de internet. La segunda manera es creando un proyecto con archivos de código que nos permitirán correr y modificar ciertos detalles del proyecto. En esta clase vamos a usar ambas formas; para propósitos de este ejemplo vamos a trabajar [con esta página](http://quotes.toscrape.com/). Esta cuenta con una estructura muy fácil de seguir y sirve para poder enseñar los fundamentos detrás de la utilización de scrapy.

# CSS
Una de las formas en las que Scrapy puede entrar a los índices de las páginas es por medio de CSS. Es un acrónimo para cascading style sheets y se usa mucho en el diseño web; HTML se utiliza para escribir el texto fundamental de la página mientras que CSS sirve más para las propiedades estéticas de la página. Dicho de otra forma; HTML sería como la fundación de la casa mientras que CSS es el color de la pared y la elección de estética. ¿Como se ve esto reflejado en el código de una página de internet? Depende de la forma en que se [implemente](https://skillcrush.com/2012/04/03/css/).

Ahora; esto nos sirve en Scrapy por que los elementos que se usan para cambiar propiedades en una página se pueden usar para extraer información de la misma página. Esto nos da una forma más fácil para navegar los requests.

In [ ]:
#Comenzamos inicializando un nuevo proyecto
scrapy shell "http://quotes.toscrape.com"
'''A continuacion se debería de ver mucho texto pero al final tenemos 3 flechas que indican que estamos dentro. 
Muy importante que vean que haya un 200 en response ya que eso nos dice si la página se cargó exitosamente.
Una vez dentro de la "shell" podemos usar el método de CSS para acceder a las propiedades de nuestro interés en la página.'''
#Podemos obtener el título de la página.
response.css('title')
#Para remover el xpath usamos el método extract.
response.css('title').extract()
#Si solo queremos el texto, usamos el siguiente comando.
response.css('title::text').extract()

Aunque extraer el título es útil, la información se encuentra en otros lados más interesantes. Usando css podemos navegar las propiedades div, liv, a y p con facilidad y diferenciar entre las respectivas clases.

In [ ]:
#Si quisieramos obtener la quote, está en <span class="text cosas"
response.css('span.text::text').extract()
#Para el autor vemos que está en <small class="author"
response.css('small.author::text').extract()

En este caso cuando nos queremos referir a una clase usamos el punto; sin el lugar de clase tuvieramos "id" entonces en css se usa el signo de #. Ahora, estos son fundamentos de css que se pueden usar con scrapy; sin embargo, hay veces que nos convendrá más usar Xpath o inclusive una combinación de ambos.

### Xpath

Xpath es el camino que se utiliza dentro de los documents xlm (otro formato web); este sería más conveniente para acceder objetos del tipo href u otros escondidos dentro de la página. Es de cierta forma un orden que tiene el código; dicho de otra forma; si el código de la página es la fundación de la casa y CSS son las cuestiones de estética entonces Xpath se puede ver como los planos de la casa. Puede que no todas las casas tengan estética pero todas tienen un plano; sin embargo, hoy en día se puede usar el que prefieran. A continuación se muestran los resultados del uso de xpath de forma individual e inclusive junto con css.

In [ ]:
#Para extraer el titulo usamos el siguiente código.
response.xpath('//title').extract()
#Si queremos extraer solo el texto hacemos la siguiente modificación
response.xpath('//title/text()').extract()
#Que si queremos acceder un renglón con una clase?
response.xpath('//span[@class="text"]/text()').extract()
#Ahora queremos recuperar el href del botón que nos lleva a la página 2
response.xpath('//li[@class="next"]/a/@href').extract()
#Con CSS, esto sería
response.css('li.next a::attr("href")').extract()
#Incluso se puede combinar con ambos.
response.css('li.next a').xpath('@href').extract()
#Ahora... esto se logra dentro de la terminal y el punto es probar antes de implementar...

# Implementación y automatización de Scrapy
El verdadero poder de Scrapy se hace presente cuando se usan las herramientas de forma correcta; por ahora, nuestro interés es navegar la página y probar los comandos pero lo ideal es diseñar las "spiders" para que extraigan información y guarden esta información en un formato utlilizable. Para ello; tenemos que recurrir a la programación orientada a objetos.

In [ ]:
#Primero entramos a la ventana de comandos y vamos al folder donde queremos trabajar en el proyecto. Una vez hecho...
scrapy startproject NombredeProyecto

El resultado es una carpeta con el nombre de nuestro proyecto, varios scripts y una carpeta llamada Spiders (ahi es donde almacenamos los spiders que creamos). Después entraremos en detalle acerca de lo que hace cada archivo pero por ahora no es necesario modificar ni uno. Comenzamos por crear un archivo en la carpeta de Spider y lo llamamos de una forma que lo puedan identificar.

In [ ]:
#Esto es lo que tenemos que escribir en el archivo si es que queremos scrappear el texto.
#Explicación Linea por Linea
import scrapy

class QuoteSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = ["http://quotes.toscrape.com"]

    def parse(self,response):
        title= response.css('title::text').extract()
        yield {'titletext': title}

Para correr el código necesitamos tener varios paquetes instalados (conda install twisted o pip install twisted), (conda install pywin32 o pip install pywin32). Una vez que tengan los paquetes teclean en la terminal scrapy crawl quotes; les debería de regresar un código bastante largo pero si ven bien ahí se obtiene el resultado.

# ¿Como lo guardo?

Este ejemplo fue muy sencillo ya que solo se obtuvo el título de una sola página. Si quisieramos obtener varias propiedades de esa página tendríamos que cambiar la función parse y para guardar el documento tenemos que seguir un proceso.

1. Scrappear los datos por medio de una araña.
2. Asignarlo a items temporales
3. Exportarlo.

En este caso vamos a hacer una pequeña prueba para exportar las frases, el autor y los tags de la frase. Luego las vamos a guardar en las variables temporales apropiadas para después exportarlas a un documento.

In [ ]:
import scrapy
class QuoteSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = ["http://quotes.toscrape.com"]
    
    def parse(self,response):
        all_div_quotes=response.css('div.quote')
        for quotes in all_div_quotes:
            text= quotes.css('span.text::text').extract()
            author= quotes.css('.author::text').extract()
            tags= quotes.css('.tag::text').extract()            
            yield {'text':text,'author':author,'tags':tags}

Ahora vamos a modificar uno de los archivos iniciales que se crearon, items.py . Este archivo le permite a la araña tener varios campos de datos que pueden ser llenados con información obtenida por parte de la función parse.

In [ ]:
import scrapy


class LaItem(scrapy.Item):
    text=scrapy.Field()
    author=scrapy.Field()
    tags=scrapy.Field()
    pass

Ahora solo queda cambiar el código de parse para asignarles valores a estos nuevos campos con los que cuenta la araña. El código modificado se encuentra a continuación.

In [ ]:
import scrapy
from ..items import LaItem
class QuoteSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = ["http://quotes.toscrape.com"]

    def parse(self,response):
        items=LaItem()
        all_div_quotes=response.css('div.quote')
        for quotes in all_div_quotes:
            text= quotes.css('span.text::text').extract()
            author= quotes.css('.author::text').extract()
            tags= quotes.css('.tag::text').extract()

            items['title']=title
            items['author']=author
            items['text']=text
            
            yield items

Para grabar el archivo final a un documento solo se debe de modificar el comando de entrada un poco.

-  scrapy crawl quotes -o archivo.csv
-  scrapy crawl quotes -o archivo.json
-  scrapy crawl quotes -o archivo.xml

Esto lo que hace es que los items resultantes se guardan en un archivo que se puede utilizar como una base de datos.

# Funciones Lambda, List Comprehension, Map, Filter y Reduce
Hay ocasiones antes, mientras y después de que obtenemos los datos en donde necesitamos hacer transformaciones o filtrado de los datos en listas. Esto se puede lograr con un iterador (for, while) en donde se hace una asignación en cada ciclo; sin embargo, Python ofrece muchas herramientas para el filtrado directo de los elementos usando métodos nativos. Estos métodos se les conoce como mapreduce methods por que dos de ellos comparten esos nombres "Map" y "Reduce". Ahora; estos métodos difieren de los otros que hemos visto por que ahora toman como parámetro de entrada una función; esto generalmente implica un proceso tardado que ocupa espacio para generar una función que probablemente nunca volveremos a utilizar. Por lo tanto; es aqui donde nos beneficia bastante utilizar funciones lambda.

## Funciones Lambda
Una función lambda es una función que se puede y debe de escribir en una línea; esta es una función anónima y no necesita de un nombre para identificarla aunque se le puede asignar uno.

In [39]:
def función(x):
    y=x+1
    return y
F=lambda x: x+1
print(F(1))
F=lambda x,y: x+y
F(2,2)

2


4

In [1]:
#Ahora, hagan una función anónima que reciba un número y regrese su cuadrado.
lambda x: x**2
F=lambda x: 'Pastel'==x
F('Pastel')

True

## Map
El método map nos sirve para transformar una serie de datos a otros datos que son de nuestro interés. Es una transformación de uno a uno (la lista final tiene que tener la misma longitud que la lista inicial). Esto nos sirve cuando hay elementos numéricos dentro de una lista y los queremos multiplicar sin el uso de paquetes como numpy; asimismo lo podemos usar para aplicar una operación a una larga lista de strings.

In [3]:

Precios=[10.00,5.00,100.00,50.00,30.00,29.00,22.00]
for precio in range(0,len(Precios)):
    Precios[precio]=Precios[precio]*2
print(Precios)

Precios=[10.00,5.00,100.00,50.00,30.00,29.00,22.00]
print(list(map(lambda x: x*2, Precios)))


[20.0, 10.0, 200.0, 100.0, 60.0, 58.0, 44.0]
[20.0, 10.0, 200.0, 100.0, 60.0, 58.0, 44.0]


In [11]:
Nombres=['Ana','Beto','Carlos','Diana','Esteban','Fabio']
for i in range(0,len(Nombres)):
    Nombres[i]=Nombres[i].lower()
print(Nombres)

Nombres=['Ana','Beto','Carlos','Diana','Esteban','Fabio']
print(list(map(lambda x: x.lower(),Nombres)))


['ana', 'beto', 'carlos', 'diana', 'esteban', 'fabio']
['ana', 'beto', 'carlos', 'diana', 'esteban', 'fabio']


# Filter
La función filter tiene los mismos parámetros de entrada que la función Map; pero su salida no cuenta con una correspondencia uno a uno. Esta función solo deja salir los elementos de la lista que regresen True al ser introducidas en la función lambda lo cual nos permite descartar ciertos errores en nuestra base de datos o eliminar precios en un rango indeseable.

In [43]:
Precios=[10.00,5.00,100.00,50.00,30.00,29.00,22.00]
Preciosfilt=[]
for i in range(0,len(Precios)):
    if Precios[i] < 50:
        Preciosfilt.append(Precios[i])
print(Preciosfilt)


Precios=[10.00,5.00,100.00,50.00,30.00,29.00,22.00]
print(list(filter(lambda x: x<50,Precios)))


[10.0, 5.0, 30.0, 29.0, 22.0]
[10.0, 5.0, 30.0, 29.0, 22.0]


In [23]:
Nombres=['Ana','Beto','','','Carlos','Diana','Esteban','','','Fabio','']
Nombresfilt=[]
for i in range(0,len(Nombres)):
    if Nombres[i] != '':
        Nombresfilt.append(Nombres[i])
print(Nombresfilt)

Nombres=['Ana','Beto','','','Carlos','Diana','Esteban','','','Fabio','']
print(list(filter(lambda x: x != '',Nombres)))

['Ana', 'Beto', 'Carlos', 'Diana', 'Esteban', 'Fabio']
['Ana', 'Beto', 'Carlos', 'Diana', 'Esteban', 'Fabio']


## Reduce
Reduce nos permite hacer una operación a todos los elementos en la lista; es decir, al inicio toma los 2 valores de la secuencia y hace una operación entre ellos; luego toma el resultado y hace una operación entre ese resultado y el siguiente valor de la secuencia. Esto nos sirve si queremos sumar todos los elementos de una lista o si queremos multiplicar todos los elementos entre ellos.

In [6]:
from functools import reduce
Frase=['Anita','lava','la','tina.']
Oracion=''
for palabra in Frase:
    Oracion=Oracion+palabra+' '
print(Oracion)
L=lambda x,y: x+' '+y
print(reduce(L,Frase))

Anita lava la tina. 
Anita lava la tina.


In [8]:
Numeros=[1,2,3,4,5,6,7,8,9,10]
Suma=0
for numero in Numeros:
    Suma=Suma+numero
print(Suma)
S=lambda x,y: x+y
print(reduce(S,Numeros))

55
55


## List comprehensions.
Una forma más fácil todavia de hacer filtrado y mapeo es por medio de comprensiones de lista. Es como un for pero escrito en un renglón y optimizado en la asignación de elementos de la lista; de hecho, este es el método más avanzado y el más usado para hacer el filtrado de listas, tuplas o diccionarios. Para hacer una comprensión de lista se comienza por definir una variable que almacenará el resultado y después se abre paréntesis.

In [10]:
#Elevar todos los números al cuadrado
Square=[numero**2 for numero in Numeros]
print(Square)
#Filtrar números
Squarefilt=[Num for Num in Square if Num<10]
print(Squarefilt)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[1, 4, 9]


Esto se puede aplicar a cualquier iterable de muchas dimensiones por lo cual se pueden anidar varios fors y condionales dentro de la comprensión pero no siempre se verá tan bonito. En esos casos es mejor definir una función que esconda las comprensiones de lista; nosotros podemos poner varios fors anidados en la comprensión e inclusive condicionales.

In [11]:
Palabras=['Juan','','Pedro','de','la','']
Palabras=[Palabra if Palabra != '' else 'ERROR' for Palabra in Palabras]
print(Palabras)

['Juan', 'ERROR', 'Pedro', 'de', 'la', 'ERROR']


Ahora bien; también se pueden crear diccionarios y tuplas con el mismo método de comprensión si es que lo prefieren.

In [50]:
Tuplecomp=tuple((i**2 for i in range(0,10)))
print(Tuplecomp)
List=['A','B','C','D','E','G']
Dictcomp={j: i for i,j in enumerate(List) }
print(Dictcomp)

(0, 1, 4, 9, 16, 25, 36, 49, 64, 81)
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5}
